In [2]:
# Connecting to the db
import lib.handle_db as dbh

# read and write csv files
import lib.handle_csv as csv_rw

# date functions
from datetime import datetime, date, timedelta

#CR libraries
from crossref.restful import Works, Etiquette

# search for UKCH Awards in CR record
def award_in_crossref(aw, award_list):
    ukch_wks =[]
    not_revised = []
    for wk in aw:      
        awd_list = []
        if 'funder' in wk.keys():
            for fdr in wk['funder']:
                if 'award' in fdr.keys():
                   awds = 0
                   for awd in fdr['award']:
                        if awd in award_list:
                            awd_list.append(awd)
                            #print (fdr)
                            break
        else:
            not_revised.append(wk)
        if len(awd_list) > 0:
            ukch_wks.append(wk)
    return ukch_wks, not_revised

def collect_keys(aw, wk_keys):
    for wk in aw:
        these_keys = wk.keys()
        for a_key in these_keys:
            if a_key in wk_keys:
                wk_keys[a_key] += 1
            else:
                wk_keys[a_key] = 1
    return wk_keys

def collect_fdr_keys(aw, wk_keys):
    for wk in aw:
        if 'funder' in wk.keys():
            for fdr in wk['funder']:
                these_keys = fdr.keys()
                for a_key in these_keys:
                    if a_key in wk_keys:
                        wk_keys[a_key] += 1
                    else:
                        wk_keys[a_key] = 1
    return wk_keys

def collect_awds(aw, wk_keys):
    for wk in aw:
        if 'funder' in wk.keys():
            for fdr in wk['funder']:
                if 'award' in fdr.keys():
                   awds = 0
                   for awd in fdr['award']:
                        if awd in wk_keys:
                            wk_keys[awd] += 1
                        else:
                            wk_keys[awd] = 1
    return wk_keys

# search for UKCH Affiliation in CR record
def affi_in_crossref(aw):
    ukch_wks = []
    for wk in aw:
        ukch_affiliation = False
        if 'author' in wk.keys():
            for autr in wk['author']:
                if 'affiliation' in autr.keys():
                    for affi in autr['affiliation']:
                        if "UK Catalysis Hub" in affi['name'] or "Catalysis Hub" in affi['name']:
                            ukch_affiliation = True
                            break
                    if ukch_affiliation:
                        ukch_wks.append(wk)
                        break
    return  ukch_wks

In [7]:
doi_list = ['10.1002/adfm.202302102', '10.1002/adma.202205153', '10.1002/admi.202101241', '10.1002/advs.202204488', 
            '10.1002/aic.16410', '10.1002/anie.201504227', '10.1002/anie.201913660', '10.1002/anie.202103805', 
            '10.1002/anie.202309305', '10.1002/anie.202312645', '10.1002/cctc.201402642', '10.1002/cctc.202000974', 
            '10.1002/cctc.202201410', '10.1002/cctc.202300870', '10.1002/chem.201701978', '10.1002/cssc.201300834', 
            '10.1007/s10563-014-9173-z', '10.1007/s11244-018-1081-4', '10.1007/s11244-020-01389-7', 
            '10.1007/s11244-020-01400-1', '10.1016/j.apcata.2015.12.031', '10.1016/j.apcatb.2020.119062', 
            '10.1016/j.apcatb.2021.120652', '10.1016/j.carbon.2016.05.065', '10.1016/j.catcom.2020.106149',
            '10.1016/j.catcom.2021.106340', '10.1016/j.catcom.2022.106415', '10.1016/j.catcom.2023.106694', 
            '10.1016/j.cattod.2014.03.001', '10.1016/j.ccst.2021.100001', '10.1016/j.ccst.2022.100095',
            '10.1016/j.cej.2022.135394', '10.1016/j.cjche.2020.05.027', '10.1016/j.cjche.2020.06.025', 
            '10.1016/j.cplett.2017.10.027', '10.1016/j.electacta.2021.138811', '10.1016/j.eurpolymj.2022.111496',
            '10.1016/j.fuel.2020.119308', '10.1016/j.ijhydene.2014.11.097', '10.1016/j.jcat.2020.06.021', 
            '10.1016/j.jcat.2022.10.025', '10.1016/j.jece.2022.108972', '10.1016/j.jechem.2021.08.003', 
            '10.1016/j.micromeso.2021.110903', '10.1016/j.micromeso.2021.110912', '10.1016/j.micromeso.2021.111535', 
            '10.1016/j.micromeso.2022.112391', '10.1016/j.trechm.2020.03.002', '10.1016/S1872-2067(19)63350-4', 
            '10.1021/acs.chemrev.1c00493', '10.1021/acs.iecr.7b04697', '10.1021/acs.jctc.8b01036', 
            '10.1021/acs.jpca.2c08022', '10.1021/acs.jpca.2c080223305', '10.1021/acs.jpcc.2c08749', 
            '10.1021/acs.jpcc.5b03577', '10.1021/acs.jpcc.5b06875', '10.1021/acs.jpcc.5b07324', 
            '10.1021/acs.jpcc.8b07846', '10.1021/acs.jpcc.8b08461', '10.1021/acs.jpclett.6b02181', 
            '10.1021/acs.oprd.0c00075', '10.1021/acsami.3c04613', '10.1021/acscatal.0c01305', '10.1021/acscatal.2c03907', 
            '10.1021/acscatal.3c00924', '10.1021/acscatal.7b01646', '10.1021/acscatal.7b04115', '10.1021/acscatal.9b04819',
            '10.1021/acsnano.1c09864', '10.1021/jacs.2c12728', '10.1021/jacs.2c13567', '10.1021/jacs.3c03935', 
            '10.1021/jacs.3c03935', '10.1038/ncomms7540', '10.1038/s41570-023-00470-5', '10.1038/s41929-021-00710-1', 
            '10.1038/s41929-022-00821-3', '10.1038/srep15979', '10.1039/c4cp04488f', '10.1039/c4cp04898a', 
            '10.1039/c5nj00804b', '10.1039/c6cc90049f', '10.1039/c6cp04622c', '10.1039/c6fd00202a', '10.1039/c8cy90033g', 
            '10.1039/c8ra10509j', '10.1039/c9cy01548e', '10.1039/d0cp04472e', '10.1039/d0cp90186e', '10.1039/d0cy00670j', 
            '10.1039/d1cp01602d', '10.1039/d1cp02535j', '10.1039/d2cp01019d', '10.1039/d2cy01699k', '10.1039/d2ta03837d', 
            '10.1039/d2ta04485d', '10.1039/d3cp02493h', '10.1039/d3cp90127k', '10.1039/d3cy00289f', '10.1039/d3re00195d',
            '10.1063/1.4952837', '10.1073/pnas.1918936117', '10.1080/2055074X.2017.1278870', 
            '10.1080/2055074X.2017.1278890', '10.1088/1361-648X/abfe16', '10.1088/1361-648X/ac0476', 
            '10.1088/1361-648X/ac1301', '10.1088/1361-648X/ace01a', '10.1098/rsos.211516', '10.1098/rspa.2022.0201', 
            '10.1098/rsta.2022.0335', '10.1098/rsta.2022.0337', '10.1098/rsta.2022.0355', '10.1098/rsta.2023.0175', 
            '10.1107/S1600576715020701', '10.1111/jace.19216', '10.1126/sciadv.abl8675', '10.1128/aem.02175-22', 
            '10.1140/epjb/e2017-80280-7', '10.1149/1945-7111/ab9e3c', '10.1162/dint_a_00143', 
            '10.1179/2055075814Y.0000000005', '10.1179/2055075814Y.0000000006', '10.1179/2055075815Y.0000000001', 
            '10.1179/2055075815Y.0000000002', '10.1179/2055075815Y.0000000008', '10.1595/205651316X691230', 
            '10.2138/am-2021-7752', '10.2138/gselements.17.3.175', '10.3389/fceng.2022.1027167', '10.3390/catal10010011', 
            '10.3390/ma13245676']



my_etiquette = Etiquette('UK Catalysis Hub Catalysis Data Infrastructure', 'Prototype 1', 'https://ukcatalysishub.co.uk/core/', 'nievadelahidalgaa@cardiff.ac.uk')

ukch_awards = ['EP/R026939/1', 'EP/R026815/1', 'EP/R026645/1', 'EP/R027129/1', 'EP/M013219/1',
               'EP/K014706/2', 'EP/K014668/1', 'EP/K014854/1', 'EP/K014714/1',]

ukch_affi_synonyms = ["UK Catalysis Hub"]

mcc_awards =['EP/R029431','EP/P020194', 'EP/T022213', 'EP/D504872', 'EP/F067496',]

award_list = ukch_awards + mcc_awards

works = Works(etiquette=my_etiquette)

start_date = end_date = date(2023, 9, 1) #started 7,1
stop_date = date(2023, 10, 15)

out_dir = './WebOfScience/'

In [8]:
pubs_with_award = []
skiped_works =[]
wk_keys = {}
fd_keys = {}
awds_lst = {}

foud_pubs = {}

for a_doi in doi_list:
    print ("Looking for ", a_doi)
    pub_w_grant = works.filter(doi=a_doi)
    for wk in pub_w_grant:
        mcc_pub = ukch_pub = False
        art_authors = ""
        
        ukch_affiliation = False
        if 'author' in wk.keys() :
            for autr in wk['author']:
                if art_authors == "":
                    art_authors = autr['family'] + (", "+ autr ['given'] if 'given' in autr.keys() else "" )
                else:
                    art_authors += ", " + autr['family']+ (", "+ autr ['given'] if 'given' in autr.keys() else "" )
                if 'affiliation' in autr.keys() and ukch_affiliation == False:
                    for affi in autr['affiliation']:
                        if "UK Catalysis Hub" in affi['name'] or "Catalysis Hub" in affi['name']:
                            ukch_affiliation = True
                            break
        fund_award = ""
        if 'funder' in wk.keys():
            for fdr in wk['funder']:
                if 'award' in fdr.keys():
                  for awd in fdr['award']:
                       if awd in award_list:
                            if fund_award  == "":
                                fund_award = awd
                            else:
                                fund_award += ", " +awd
                            if awd in ukch_awards:
                                ukch_pub = True
                            if awd in mcc_awards:
                                mcc_pub = True
        ol_year = 0
        pr_year = 0
        pub_year = 0
        if 'published-online' in wk.keys() and 'date-parts' in wk['published-online'].keys():
            ol_year = int(wk['published-online']['date-parts'][0][0])
        if 'published-print' in wk.keys() and 'date-parts' in wk['published-print'].keys():
            pr_year = int(wk['published-print']['date-parts'][0][0])
        if pr_year > 0 and ol_year > 0:
            if pr_year > ol_year:
                pub_year = ol_year
            else:
                pub_year = pr_year
        elif ol_year > 0:
            pub_year = ol_year
        elif pr_year > 0:
            pub_year = pr_year

        print(art_authors,"|",pub_year,"|",wk['title'][0],
              "|", wk['DOI'],"|", fund_award)     
        this_pub = {}
        this_pub['authors'] = art_authors
        this_pub['year'] = pub_year
        this_pub['title'] = wk['title'][0]
        this_pub['DOI'] = wk['DOI']
        this_pub['awards'] = fund_award

        this_pub['mcc'] = 1 if mcc_pub else 0
        this_pub['ukch'] = 1 if ukch_pub else 0
        this_pub['ukch_affiliation'] = 1 if ukch_affiliation else 0
        if not wk['DOI'] in foud_pubs:
             foud_pubs[wk['DOI']]= this_pub

        
# WRITE TO FILE
if len(foud_pubs) > 0:
    #csv_rw.write_csv_data(foud_pubs, out_dir+'cr_check_'+str(end_date)+'a.csv') 
    print(foud_pubs)
        

Looking for  10.1002/adfm.202302102
Wang, Mei, Papaioannou, Evangelos I., Metcalfe, Ian S., Naden, Aaron, Savaniu, Cristian D., Irvine, John T. S. | 2023 | The Exsolution of Cu Particles from Doped Barium Cerate Zirconate via Barium Cuprate Intermediate Phases | 10.1002/adfm.202302102 | 
Looking for  10.1002/adma.202205153
Pi, Yutong, Ma, Yanfu, Wang, Xinyao, Price, Cameron‐Alexander Hurd, Li, Haitao, Liu, Qinglong, Wang, Liwei, Chen, Hongyu, Hou, Guangjin, Su, Bao‐Lian, Liu, Jian | 2022 | Multilevel Hollow Phenolic Resin Nanoreactors with Precise Metal Nanoparticles Spatial Location toward Promising Heterogeneous Hydrogenations | 10.1002/adma.202205153 | 
Looking for  10.1002/admi.202101241
Li, Xinyuan, Xu, Yikai, Greer, Adam J., Yue, Mingjian, Ye, Ziwei, McNeill, Aaron, Li, Chunchun, Mills, Andrew, Hu, Peijun, Hardacre, Christopher, Bell, Steven E. J. | 2021 | A Simple and Ligand‐Free Synthesis of Light and Durable Metal‐TiO<sub>2</sub> Polymer Films with Enhanced Photocatalytic Prop

Looking for  10.1016/j.catcom.2020.106149
Qiu, Chengwu, Meng, Qingwei, Panchal, Monik, Li, Changzhi, Wu, Baoshan | 2020 | Enhanced Fischer-Tropsch activity in ammonium nitrate pretreated cobalt-silica catalyst | 10.1016/j.catcom.2020.106149 | 
Looking for  10.1016/j.catcom.2021.106340
Campisi, Sebastiano, Capelli, Sofia, Ferri, Michele, Villa, Alberto, Dann, Ellie, Wade, Austin, Wells, Peter P., Dimitratos, Nikolaos | 2021 | On the role of bismuth as modifier in AuPdBi catalysts: Effects on liquid-phase oxidation and hydrogenation reactions | 10.1016/j.catcom.2021.106340 | 
Looking for  10.1016/j.catcom.2022.106415
Woodward, Claire-Louise M., Porter, Alexander J., Morton, Katie S.C., O'Malley, Alexander J. | 2022 | Methanol diffusion in H-ZSM-5 catalysts as a function of loading and Si/Al ratio: A classical molecular dynamics study | 10.1016/j.catcom.2022.106415 | 
Looking for  10.1016/j.catcom.2023.106694
Hayward, James, Esquius, Jonathan Ruiz, Richards, Nia, Jones, Wilm, Hewes, Danie

Looking for  10.1021/acs.jpca.2c08022
Morton, Katie S. C., Elena, Alin M., Armstrong, Jeff, O’Malley, Alexander J. | 2023 | Experimental and Modeling Studies of Local and Nanoscale <i>para</i>-Cresol Behavior: A Comparison of Classical Force Fields | 10.1021/acs.jpca.2c08022 | 
Looking for  10.1021/acs.jpca.2c080223305
Looking for  10.1021/acs.jpcc.2c08749
Britto, Sylvia, Parlett, Christopher M. A., Bartlett, Stuart, Elliott, Joshua D., Ignatyev, Konstantin, Schroeder, Sven L. M. | 2023 | Intermediates during the Nucleation of Platinum Nanoparticles by a Reaction with Ethylene Glycol: Operando X-ray Absorption Spectroscopy Studies with a Microfluidic Cell | 10.1021/acs.jpcc.2c08749 | 
Looking for  10.1021/acs.jpcc.5b03577
Gould, Anna L., Logsdail, Andrew J., Catlow, C. Richard A. | 2015 | Influence of Composition and Chemical Arrangement on the Kinetic Stability of 147-Atom Au–Ag Bimetallic Nanoclusters | 10.1021/acs.jpcc.5b03577 | 
Looking for  10.1021/acs.jpcc.5b06875
Lezcano-Gonzale

Looking for  10.1038/s41929-021-00710-1
Avanthay, Mickaël, Bedford, Robin B., Begg, Callum S., Böse, Dietrich, Clayden, Jonathan, Davis, Sean A., Eloi, Jean-Charles, Goryunov, Georgy P., Hartung, Ingo V., Heeley, Joseph, Khaikin, Kirill A., Kitching, Matthew O., Krieger, Johannes, Kulyabin, Pavel S., Lennox, Alastair J. J., Nolla-Saltiel, Roberto, Pridmore, Natalie E., Rowsell, Benjamin J. S., Sparkes, Hazel A., Uborsky, Dmitry V., Voskoboynikov, Alexander Z., Walsh, Mark P., Wilkinson, Harry J. | 2021 | Identifying palladium culprits in amine catalysis | 10.1038/s41929-021-00710-1 | 
Looking for  10.1038/s41929-022-00821-3
Bell, Elizabeth L., Smithson, Ross, Kilbride, Siobhan, Foster, Jake, Hardy, Florence J., Ramachandran, Saranarayanan, Tedstone, Aleksander A., Haigh, Sarah J., Garforth, Arthur A., Day, Philip J. R., Levy, Colin, Shaver, Michael P., Green, Anthony P. | 2022 | Directed evolution of an efficient and thermostable PET depolymerase | 10.1038/s41929-022-00821-3 | 
Looking

Looking for  10.1073/pnas.1918936117
Kwon, Hanna, Basran, Jaswir, Devos, Juliette M., Suardíaz, Reynier, van der Kamp, Marc W., Mulholland, Adrian J., Schrader, Tobias E., Ostermann, Andreas, Blakeley, Matthew P., Moody, Peter C. E., Raven, Emma L. | 2020 | Visualizing the protons in a metalloenzyme electron proton transfer pathway | 10.1073/pnas.1918936117 | 
Looking for  10.1080/2055074X.2017.1278870
Parker, Stewart F., Carias, Celine, Tomkinson, John | 2017 | Vibrational spectra of buta-1,3-diene iron tricarbonyl: comparison to surface species | 10.1080/2055074x.2017.1278870 | 
Looking for  10.1080/2055074X.2017.1278890
Gibson, E. K., Crabb, E. M., Gianolio, D., Russell, A. E., Thompsett, D., Wells, P. P. | 2017 | Understanding the role of promoters in catalysis: <i>operando</i> XAFS/DRIFTS study of CeO<sub>x</sub>/Pt/Al<sub>2</sub>O<sub>3</sub> during CO oxidation | 10.1080/2055074x.2017.1278890 | EP/K014714/1, EP/K014854/1
Looking for  10.1088/1361-648X/abfe16
Panchal, Monik, Call

Looking for  10.2138/gselements.17.3.175
Benmore, Chris J., Wilding, Martin C. | 2021 | Probing the Structure of Melts, Glasses, and Amorphous Materials | 10.2138/gselements.17.3.175 | 
Looking for  10.3389/fceng.2022.1027167
Charalambous, Christina, Xu, Shanshan, Ding, Shengzhe, Chansai, Sarayute, Asuquo, Edidiong, Torres Lopez, Antonio, Parlett, Christopher M. A., Gilmour, Jamie D., Garforth, Arthur, Hardacre, Christopher | 2022 | Non-thermal plasma activated CO2 hydrogenation over K- and La- promoted layered-double hydroxide supported Ni catalysts | 10.3389/fceng.2022.1027167 | 
Looking for  10.3390/catal10010011
Alijani, Shahram, Capelli, Sofia, Cattaneo, Stefano, Schiavoni, Marco, Evangelisti, Claudio, Mohammed, Khaled M. H., Wells, Peter P., Tessore, Francesca, Villa, Alberto | 2019 | Capping Agent Effect on Pd-Supported Nanoparticles in the Hydrogenation of Furfural | 10.3390/catal10010011 | 
Looking for  10.3390/ma13245676
Jones, Matthew P., Storm, Malte, York, Andrew P. E., Hy

In [9]:
if len(foud_pubs) > 0:
    csv_rw.write_csv_data(foud_pubs, out_dir+'cr_check_wos_2023b.csv') 